In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as d_copy
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#error_index = 0
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)#.to(device)
print(device)

cuda


In [3]:
from importnb import Notebook

In [4]:
with Notebook(lazy=True):
    import F4F_model
header = F4F_model

In [5]:
# set randomness
seed = 0

header.set_randomness(seed) # ipynb module import means run all cell in file
# load dataset
dataset_path = "/media/2/Network/Imagenet_dup/"
retrain_model_path = "/media/0/Network/0722_to_fullmodels/"
batch_size = 32 # 32~ out of memory in 3080
num_train = 640000

train_dataloader,test_dataloader = header.get_dataset(num_train,batch_size,
                  dataset_path,retrain_model_path)

===INFO===
torch ver : 1.8.0
torchvision ver : 0.2.2 
GPU model : TITAN RTX
anaconda3	   environment.yml  jupyter.sh	todolist
docker_script.txt  F4F		    pastebin
train dataset[20000], test dataset[1563] are loaded


In [6]:
#In_layer_number = 34 # 34 conv5_1 convolution
#Out_layer_number = 36 # 36 conv5_1 relu 
error_index=0
max_epochs = 30
num_error = 128

In [7]:
log_file = "./128_acc_log_toEnd_0801.txt"

In [8]:
#split_model = split_layer(vgg16_bn,0,Out_layer_number)

original_model = d_copy(vgg16_bn).to(device)
# subset of vgg16 (whole layer) with f4f
header.hook_register(vgg16_bn,error_index,num_error)
target_model = header.Target_model(vgg16_bn).to(device)



In [9]:
f4f = header.F4F().to(device)
optimizer = torch.optim.SGD(f4f.parameters(),lr=0.1e-7,weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()


In [ ]:

#optimizer = torch.optim.SGD(param_list,lr=0.01,weight_decay=1e-4)
first_feature = []
original_out = []
f = open(log_file,"w")
print("reverse sequence of error_index & filter_set")
f.close()
num_error = 128
max_epoch = 50
for epoch in range(max_epoch):
    print("======= epoch %2d ======="%(epoch))
    header.hook_register(vgg16_bn,error_index,num_error)
    target_model = header.Target_model(vgg16_bn).to(device)
    
    for error_idx in range(0,512,num_error):
        print("error(%d~%d) inserted training"%(error_idx,error_idx+num_error))
        tmp= header.training(f4f,target_model,original_model,
                  train_dataloader,test_dataloader,
                  log_file,retrain_model_path,
                  loss_fn,optimizer,
                  error_idx,num_error,1,True)
    first_feature.append(tmp[1])
    original_out.append(tmp[0])

reverse sequence of error_index & filter_set
======= epoch  0 =======
error(0~128) inserted training
..........
[1,   100] loss: 7.837015
..........
[1,   200] loss: 7.675049
..........
[1,   300] loss: 7.569635
..........
[1,   400] loss: 7.562634
..........
[1,   500] loss: 7.515431
..........
[1,   600] loss: 7.503632
..........
[1,   700] loss: 7.471245
..........
[1,   800] loss: 7.482696
..........
[1,   900] loss: 7.517287
..........
[1,  1000] loss: 7.492352
..........
[1,  1100] loss: 7.470495
..........
[1,  1200] loss: 7.479049
..........
[1,  1300] loss: 7.466379
..........
[1,  1400] loss: 7.485488
..........
[1,  1500] loss: 7.485773
...

In [ ]:
# 여기서부터는 feature 그림 보기 위한 것들입니다.


In [ ]:

w = 10
h = 10
cols = 32
rows = 16
def feature_print(pic):
    print("test with 'after pooling 4 feature'")
    fig = plt.figure(figsize=(64,32))
    ax = []
    for i in range(cols*rows):
        ch = pic[i,:,:]
        ax.append(fig.add_subplot(rows,cols,i+1))
        ax[-1].set_title(str(i)+"th ch (14x14)")
        plt.imshow(ch)